In [1]:
import boto3
import pandas as pd
from io import StringIO

s3 = boto3.client('s3')
BUCKET_NAME = 'sagemaker-crawling-s3-bucket'
DIR_NAME = 'krx/' #저장된 폴더명

In [124]:
#저장된 csv파일 모두 가져오기(1000개 이상의 객체 제한을 고려한 코드)
def list_all_objects(bucket_name, prefix=''):
    s3 = boto3.client('s3')
    objects = []
    continuation_token = None

    while True:
        if continuation_token:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, ContinuationToken=continuation_token)
        else:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

        # 객체 추가
        objects.extend(response.get('Contents', []))

        # 다음 토큰 확인
        if response.get('NextContinuationToken'):
            continuation_token = response['NextContinuationToken']
        else:
            break

    return objects

all_objects = list_all_objects(BUCKET_NAME, prefix=DIR_NAME)
print(len(all_objects)) #전체 데이터 수

2781


In [125]:
#.csv 파일을 읽어와서 데이터프레임으로 변환/ dfs 딕셔너리에 파일명 저장
dfs = {}
for obj in all_objects:
    file_key = obj['Key']
    if file_key.endswith('.csv'):
        file_content = s3.get_object(Bucket=BUCKET_NAME, Key=file_key)['Body'].read().decode('utf-8')
        df = pd.read_csv(StringIO(file_content))
        file_name = file_key.split('/')[-1].split('.')[0]
        dfs[file_name] = df

In [126]:
a = dfs[list(dfs.keys())[0]]
a

,TRD_DD,TDD_CLSPRC,FLUC_TP_CD,CMPPREVDD_PRC,FLUC_RT,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP,LIST_SHRS,ISU_NM
0,2023/11/28,120,1,2,1.69,130,131,120,"22,582,385","2,827,614,840","35,031,846,000","291,932,050",이스트아시아홀딩스
1,2023/11/27,118,3,0,0.00,118,119,117,"3,220,959","391,740,172","34,447,981,900","291,932,050",이스트아시아홀딩스
2,2023/11/24,118,2,-1,-0.84,120,120,118,"1,756,771","207,874,027","34,447,981,900","291,932,050",이스트아시아홀딩스
3,2023/11/23,119,2,-1,-0.83,120,120,118,"1,628,722","194,059,901","34,739,913,950","291,932,050",이스트아시아홀딩스
4,2023/11/22,120,2,-1,-0.83,122,123,119,"3,073,359","369,985,034","35,031,846,000","291,932,050",이스트아시아홀딩스
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3350,2010/04/29,"3,460",2,-240,-6.49,"3,700","3,740","3,430","1,170,700","4,157,244,845","79,580,000,000","23,000,000",이스트아시아홀딩스
3351,2010/04/28,"3,700",2,-50,-1.33,"3,700","3,970","3,660","1,422,838","5,420,720,915","85,100,000,000","23,000,000",이스트아시아홀딩스
3352,2010/04/27,"3,750",2,-110,-2.85,"3,885","3,930","3,745","1,385,013","5,268,663,780","86,250,000,000","23,000,000",이스트아시아홀딩스
3353,2010/04/26,"3,860",1,35,0.92,"3,655","4,055","3,650","4,312,630","16,652,290,815","88,780,000,000","23,000,000",이스트아시아홀딩스


In [127]:
a['MKTCAP'] = a['MKTCAP'].str.replace(',', '').astype(float)
a['TRD_DD'] = pd.to_datetime(a['TRD_DD'])

In [128]:
a

,TRD_DD,TDD_CLSPRC,FLUC_TP_CD,CMPPREVDD_PRC,FLUC_RT,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP,LIST_SHRS,ISU_NM
0,2023-11-28,120,1,2,1.69,130,131,120,"22,582,385","2,827,614,840",3.503185e+10,"291,932,050",이스트아시아홀딩스
1,2023-11-27,118,3,0,0.00,118,119,117,"3,220,959","391,740,172",3.444798e+10,"291,932,050",이스트아시아홀딩스
2,2023-11-24,118,2,-1,-0.84,120,120,118,"1,756,771","207,874,027",3.444798e+10,"291,932,050",이스트아시아홀딩스
3,2023-11-23,119,2,-1,-0.83,120,120,118,"1,628,722","194,059,901",3.473991e+10,"291,932,050",이스트아시아홀딩스
4,2023-11-22,120,2,-1,-0.83,122,123,119,"3,073,359","369,985,034",3.503185e+10,"291,932,050",이스트아시아홀딩스
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3350,2010-04-29,"3,460",2,-240,-6.49,"3,700","3,740","3,430","1,170,700","4,157,244,845",7.958000e+10,"23,000,000",이스트아시아홀딩스
3351,2010-04-28,"3,700",2,-50,-1.33,"3,700","3,970","3,660","1,422,838","5,420,720,915",8.510000e+10,"23,000,000",이스트아시아홀딩스
3352,2010-04-27,"3,750",2,-110,-2.85,"3,885","3,930","3,745","1,385,013","5,268,663,780",8.625000e+10,"23,000,000",이스트아시아홀딩스
3353,2010-04-26,"3,860",1,35,0.92,"3,655","4,055","3,650","4,312,630","16,652,290,815",8.878000e+10,"23,000,000",이스트아시아홀딩스


In [134]:
b = a[a['TRD_DD'] == '2010-04-29'].index[0]
b

3350

In [137]:
c = a.iloc[b: b+5]
c

,TRD_DD,TDD_CLSPRC,FLUC_TP_CD,CMPPREVDD_PRC,FLUC_RT,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP,LIST_SHRS,ISU_NM
3350,2010-04-29,"3,460",2,-240,-6.49,"3,700","3,740","3,430","1,170,700","4,157,244,845",7.958000e+10,"23,000,000",이스트아시아홀딩스
3351,2010-04-28,"3,700",2,-50,-1.33,"3,700","3,970","3,660","1,422,838","5,420,720,915",8.510000e+10,"23,000,000",이스트아시아홀딩스
3352,2010-04-27,"3,750",2,-110,-2.85,"3,885","3,930","3,745","1,385,013","5,268,663,780",8.625000e+10,"23,000,000",이스트아시아홀딩스
3353,2010-04-26,"3,860",1,35,0.92,"3,655","4,055","3,650","4,312,630","16,652,290,815",8.878000e+10,"23,000,000",이스트아시아홀딩스
3354,2010-04-23,"3,825",5,-675,-15.00,"4,500","4,500","3,825","6,399,130","25,724,307,950",8.797500e+10,"23,000,000",이스트아시아홀딩스


In [157]:
import pandas as pd
import numpy as np

def calculate_mdd(data, specified_date):
    """
    Calculate Maximum Drawdown (MDD) for the 5 days before the specified date.

    Parameters:
    - data: DataFrame with 'Date' and 'Close' columns.
    - specified_date: Date for which MDD is calculated.

    Returns:
    - mdd: Maximum Drawdown.
    """
    
    specified_index = data[data['TRD_DD'] == specified_date].index[0]
    previous_data = data.iloc[specified_index: specified_index + 5]

    peak_price = previous_data['MKTCAP'].max()
    previous_data['Drawdown'] = (peak_price - previous_data['MKTCAP']) / peak_price

    # 5. 최대 Drawdown 찾아 MDD 계산
    mdd = previous_data['Drawdown'].max()

    return mdd

In [158]:
specified_date = '2023-11-28'
mdd = calculate_mdd(a, specified_date)
print(f'MDD for the 5 days before {specified_date}: {mdd:.2%}')

MDD for the 5 days before 2023-11-28: 1.67%


/tmp/ipykernel_12461/3733194562.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  previous_data['Drawdown'] = (peak_price - previous_data['MKTCAP']) / peak_price
